In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import altair as alt
import dtmapi

In [2]:
all_country_list = ['Afghanistan', 'Democratic Republic of the Congo', 'Ethiopia', 'Iraq', 'Nigeria', 'Somalia', 'South Sudan', 'Sudan', 'Yemen']

# Initialize an empty list to store dataframes
all_dataframes = []

# Loop through each country in the list
for country in all_country_list:
    # Fetch data for each country
    df = dtmapi.get_idp_admin0_data(CountryName=country, FromReportingDate='2014-01-01', ToReportingDate='2024-12-31', to_pandas=True)
    all_dataframes.append(df)

dtm_df = pd.concat(all_dataframes, ignore_index=True)


In [3]:
# Convert reportingDate to datetime
dtm_df['reportingDate'] = pd.to_datetime(dtm_df['reportingDate'])
dtm_df['year'] = dtm_df['reportingDate'].dt.year
idx = dtm_df.groupby(['admin0Name', 'year'])[['monthReportingDate', 'roundNumber']].idxmax()
most_recent_data_per_year = dtm_df.loc[idx['monthReportingDate']]
most_recent_data_per_year.reset_index(drop=True, inplace=True)

# filter the top 10 countries with the highest number of IDPs
countries = most_recent_data_per_year.groupby('admin0Name')['numPresentIdpInd'].sum().nlargest(10).index
countries_data = most_recent_data_per_year[most_recent_data_per_year['admin0Name'].isin(countries)]

#remove row COD with 2018 2 3 
countries_data = countries_data[~((countries_data['admin0Name'] == 'Democratic Republic of The Congo') &(countries_data['roundNumber'] == 30))]

In [4]:
#save as csv
countries_data.to_csv('top_countries_data.csv', index=False)
#countries_data.to_json('countries_data.json', orient='records')

In [5]:
# Save the chart as a JSON file
chart_json = alt.Chart(countries_data).mark_line().encode(
    x=alt.X('yearReportingDate', axis=alt.Axis(title=None, labelAngle=360)),
    y=alt.Y('numPresentIdpInd', axis=alt.Axis(title=None)),
    color=alt.Color('admin0Name', legend=alt.Legend(title=None)),
    tooltip=['yearReportingDate', 'numPresentIdpInd', 'admin0Name'],
).properties(
    width=800,
    height=400,
    title={
        "text": "Internally displaced people by conflict over time",
        "subtitle": "Number of IDPs in the top 10 countries with the highest number of IDPs",
        "anchor": "start",
        "align": "left"
    }
).to_json()

with open('CC6_fig1.json', 'w') as f:
    f.write(chart_json)